<a href="https://colab.research.google.com/github/anirbansen3027/SecureAndPrivateAI/blob/master/11_Encrypted_Database_In_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow.keras as keras

In [0]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, AveragePooling2D,Activation

In [0]:
batch_size =128
num_classes = 10
epochs = 20

img_rows, img_cols = 28,28

In [0]:
(x_train,y_train),(x_test,y_test) = mnist.load_data()

In [0]:
x_train = x_train.reshape(x_train.shape[0],img_rows,img_cols,1)
x_test = x_test.reshape(x_test.shape[0],img_rows,img_cols,1)

In [0]:
input_shape = (img_rows,img_cols,1)
input_shape

(28, 28, 1)

In [0]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /=255
x_test /=255

In [0]:
x_train.shape,x_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [0]:
y_train = keras.utils.to_categorical(y_train,num_classes)
y_test = keras.utils.to_categorical(y_test,num_classes)

In [0]:
model = Sequential()

In [0]:
model.add(Conv2D(10,(3,3),input_shape=input_shape))
model.add(AveragePooling2D(2,2,))
model.add(Activation('relu'))
model.add(Conv2D(32,(3,3)))
model.add(AveragePooling2D(2,2,))
model.add(Activation('relu'))
model.add(Conv2D(64,(3,3)))
model.add(AveragePooling2D(2,2,))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(num_classes,activation='softmax'))
model

In [0]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
             metrics=['accuracy'])

In [0]:
model.fit(x_train,y_train,
         batch_size=batch_size,
         epochs=epochs,
         verbose=1,
         validation_data=(x_test,y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 2s 41us/sample - loss: 2.3013 - acc: 0.1141 - val_loss: 2.3001 - val_acc: 0.1221
Epoch 2/20
60000/60000 [==============================] - 2s 36us/sample - loss: 2.3002 - acc: 0.1220 - val_loss: 2.2990 - val_acc: 0.1306
Epoch 3/20
60000/60000 [==============================] - 2s 35us/sample - loss: 2.2991 - acc: 0.1286 - val_loss: 2.2978 - val_acc: 0.1378
Epoch 4/20
60000/60000 [==============================] - 2s 36us/sample - loss: 2.2981 - acc: 0.1353 - val_loss: 2.2967 - val_acc: 0.1425
Epoch 5/20
60000/60000 [==============================] - 2s 36us/sample - loss: 2.2970 - acc: 0.1394 - val_loss: 2.2956 - val_acc: 0.1464
Epoch 6/20
60000/60000 [==============================] - 2s 35us/sample - loss: 2.2959 - acc: 0.1422 - val_loss: 2.2945 - val_acc: 0.1465
Epoch 7/20
60000/60000 [==============================] - 2s 36us/sample - loss: 2.2948 - acc: 0.1434 - val_loss: 2.

In [0]:
score = model.evaluate(x_test,y_test,verbose=0)
print("Test loss:",score[0])
print("Test accuracy:",score[1])

Test loss: 2.27068505821228
Test accuracy: 0.1559


In [0]:
model.save('short-conv-mnist.h5')

In [0]:
import tensorflow as tf
import numpy as np
import syft as sy
hook = sy.KerasHook(tf.keras)

W0803 20:21:42.758044 140110791632768 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0803 20:21:42.774679 140110791632768 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
num_classes = 10
input_shape = (1, 28, 28, 1)

model = Sequential()

model.add(Conv2D(10, (3, 3), batch_input_shape=input_shape))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(num_classes, name="logit"))

In [0]:
pre_trained_weights = "short-conv-mnist.h5"
model.load_weights(pre_trained_weights)

In [0]:
AUTO = True

alice = sy.TFEWorker(host='localhost:4000',auto_managed=AUTO)
bob = sy.TFEWorker(host='localhost:4001', auto_managed=AUTO)
carol = sy.TFEWorker(host='localhost:4002', auto_managed=AUTO)

In [0]:
model.share(alice,bob,carol)

KeyError: ignored

In [0]:
!python -m tf_encrypted.player --config /tmp/tfe.config server0
!python -m tf_encrypted.player --config /tmp/tfe.config server1
!python -m tf_encrypted.player --config /tmp/tfe.config server2

W0803 20:29:34.792227 140397221697408 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0803 20:29:34.804893 140397221697408 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

Traceback (most recent call last):
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/dist-packages/tf_encrypted/player/__main__.py", line 16, in <module>
    config = RemoteConfig.load(args.config)
  File "/usr/local/lib/python3.6/dist-packages/tf_encrypted/config.py",

In [0]:
model.serve(num_requests=3)

AttributeError: ignored